# Face Recognition w/ 3D Landmarks

Melisa Mete, 150200316

Öykü Eren, 150200326

Bora Boyacıoğlu, 150200310

## Step 1: Data Preparation and Preprocessing

Import necessary libraries.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, num_classes, image_model=None, landmark_model=None, combined='Combined'):
        super(CombinedModel, self).__init__()
        self.combined = combined
        if image_model != None:
            self.image_model = image_model
        if landmark_model != None:
            self.landmark_model = landmark_model
        self.fc_ = nn.Linear(512, 512)
        self.fcom = nn.Linear(512 + 512, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, image=None, landmarks=None):
        if self.combined == 'Combined':
            image_features = self.image_model(image)
            landmark_features = self.landmark_model(landmarks)
            combined_features = torch.cat((image_features, landmark_features), dim=1)
            x = nn.ReLU()(self.fcom(combined_features))
            x = self.fc2(x)
        elif self.combined == 'Image':
            image_features = self.image_model(image)
            x = nn.ReLU()(self.fc_(combined_features))
            x = self.fc2(x)
            
        elif self.combined == 'Landmark':
            landmark_features = self.landmark_model(landmarks)
            x = nn.ReLU()(self.fc_(combined_features))
            x = self.fc2(x)
        return x

# Determine the number of classes
num_classes = 35

# Initialize the combined model
model = CombinedModel(num_classes, image_feature_extractor, landmark_extractor, combined='Combined')
model.to(device)

In [ ]:
# Train the model
num_epochs = 10
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, landmarks, labels in train_loader:
        images, landmarks, labels = images.to(device), landmarks.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, landmarks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, landmarks, labels in val_loader:
            images, landmarks, labels = images.to(device), landmarks.to(device), labels.to(device)
            outputs = model(images, landmarks)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')

# Save the trained model
torch.save(model.state_dict(), 'combined_model.pth')
